In [6]:
import json
import numpy as np
import glob
import os
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from matplotlib.colors import ListedColormap
from matplotlib import rc
import random

ModuleNotFoundError: No module named 'pandas'

In [7]:
# matplotlib 한글 폰트 깨짐 문제 해결
import matplotlib.font_manager as fm
font_path = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
fontprop = fm.FontProperties(fname=font_path)

In [8]:
# 경로 설정

data_path = '/workspace/Data_All/'

img_path = data_path + 'image/'                     # /workspace/Data_All/image/
json_path = data_path + 'label/'                    # /workspace/Data_All/label/
segment_path = data_path + 'segmentation/'          # /workspace/Data_All/segmentation/

anger_img_path = img_path + 'anger/'                 # /workspace/Data_ALL/image/anger/
happy_img_path = img_path + 'happy/'                 # /workspace/Data_ALL/image/happy/
panic_img_path = img_path + 'panic/'                 # /workspace/Data_ALL/image/panic/
sadness_img_path = img_path + 'sadness/'             # /workspace/Data_ALL/image/sadness/

In [9]:
# 라벨 정보 json 파일 (train, val, test 파일 통합)
# train_panic.json + val_panic.json + test_panic.json -> json_panic

# json 파일 각각 불러오기
with open (json_path + "train_panic.json", "r", encoding='euc-kr') as f:
    json_panic_train = json.load(f)
with open (json_path + "val_panic.json", "r", encoding='euc-kr') as f:
    json_panic_val = json.load(f)
with open (json_path + "test_panic.json", "r", encoding='euc-kr') as f:
    json_panic_test = json.load(f)

# 하나의 파일로 통합
json_panic = []
if isinstance(json_panic_train, list):
    json_panic.extend(json_panic_train)
if isinstance(json_panic_val, list):
    json_panic.extend(json_panic_val)
if isinstance(json_panic_test, list):
    json_panic.extend(json_panic_test)

In [10]:
# Segmentation 정보 npz 파일 (train, val, test 통합된 파일)
# npz_panic

npz_panic = np.load(segment_path + 'combined_panic.npz')

In [11]:
# json 파일 내 이미지 이름들과 npz 파일 내 이미지 이름들이 동일한지 확인

json_filenames = [item['filename'] for item in json_panic]
npz_filenames = list(npz_panic.keys())

# 동일 여부 확인
common_filenames = set(json_filenames) & set(npz_filenames)  # 교집합
only_in_json = set(json_filenames) - set(npz_filenames)      # JSON에만 있는 파일
only_in_npz = set(npz_filenames) - set(json_filenames)       # NPZ에만 있는 파일

# 결과 출력
print(f"공통 파일 개수: {len(common_filenames)}")
print(f"JSON에만 있는 파일 개수: {len(only_in_json)}")
print(f"NPZ에만 있는 파일 개수: {len(only_in_npz)}")

공통 파일 개수: 2100
JSON에만 있는 파일 개수: 0
NPZ에만 있는 파일 개수: 1


In [12]:
only_in_npz

{'64f21a1de3537ea5e57bfcf0195951512db6f5def5575b49eb7eea785bb25emqj(1).jpg'}

In [13]:
key_to_check_1 = '64f21a1de3537ea5e57bfcf0195951512db6f5def5575b49eb7eea785bb25emqj(1).jpg'
key_to_check_2 = '64f21a1de3537ea5e57bfcf0195951512db6f5def5575b49eb7eea785bb25emqj.jpg'

# if key_to_check_1 in npz_happy:
#     print(f"Data for key '{key_to_check_1}':")
#     print(npz_happy[key_to_check_1].shape)
#     print(npz_happy[key_to_check_1])
# else:
#     print(f"Key '{key_to_check_1}' not found in the npz file.")

# if key_to_check_2 in npz_happy:
#     print(f"Data for key '{key_to_check_2}':")
#     print(npz_happy[key_to_check_2].shape)
#     print(npz_happy[key_to_check_2])
# else:
#     print(f"Key '{key_to_check_2}' not found in the npz file.")

data1 = npz_panic[key_to_check_1]
data2 = npz_panic[key_to_check_2]

# 데이터 비교
if np.array_equal(data1, data2):
    print(f"The data for '{key_to_check_1}' and '{key_to_check_2}' are identical.")
else:
    print(f"The data for '{key_to_check_1}' and '{key_to_check_2}' are different.")

The data for '64f21a1de3537ea5e57bfcf0195951512db6f5def5575b49eb7eea785bb25emqj(1).jpg' and '64f21a1de3537ea5e57bfcf0195951512db6f5def5575b49eb7eea785bb25emqj.jpg' are identical.


In [14]:
# 겹치는 데이터 삭제

key_to_delete = key_to_check_1

updated_data = {key: npz_panic[key] for key in npz_panic.keys() if key != key_to_delete}

combined_npz_path = segment_path + 'combined_panic_updated.npz'
np.savez_compressed(combined_npz_path, **updated_data)

In [15]:
# json 파일 내 이미지 이름들과 npz 파일 내 이미지 이름들이 동일한지 확인 (npz 데이터 수정 후)

npz_panic = np.load(segment_path + 'combined_panic_updated.npz')

json_filenames = [item['filename'] for item in json_panic]
npz_filenames = list(npz_panic.keys())

# 동일 여부 확인
common_filenames = set(json_filenames) & set(npz_filenames)  # 교집합
only_in_json = set(json_filenames) - set(npz_filenames)      # JSON에만 있는 파일
only_in_npz = set(npz_filenames) - set(json_filenames)       # NPZ에만 있는 파일

# 결과 출력
print(f"공통 파일 개수: {len(common_filenames)}")
print(f"JSON에만 있는 파일 개수: {len(only_in_json)}")
print(f"NPZ에만 있는 파일 개수: {len(only_in_npz)}")

공통 파일 개수: 2100
JSON에만 있는 파일 개수: 0
NPZ에만 있는 파일 개수: 0


In [16]:
# npz 파일 확인

# 파일에 저장된 데이터 키 확인
keys = list(npz_panic.keys())[:10]  # 처음 10개의 키만 가져옴
print("npz 파일 길이 : ", len(npz_panic))
print("Keys in .npz file:", npz_panic.keys())

# 각 키에 대한 배열 정보 출력
for key in keys:
    array = npz_panic[key]
    print(f"\nKey: {key}")
    print(f"Shape: {array.shape}")
    print(f"Data type: {array.dtype}")
    print(f"Array contents:\n{array}")

npz 파일 길이 :  2100
Keys in .npz file: KeysView(NpzFile '/workspace/Data_All/segmentation/combined_panic_updated.npz' with keys: 007b10b45d424e7876e73b6b66d62d603c6ef2ee886761e570b2110e47608iwlr.jpg, 01hq73db2125c694f5f555f744b2020c112251215b9574bb0101ff04ae007ut37.jpg, 059hc8436a3e46efdc3c20e94dca8704846b408dfcfc39c2ec4a8be2ad1dazqrw.jpg, 07rq89a5d735a958d1783db31a7cd098f60969900662e10035762826bd005hlhp.jpg, 07tff4b4c1effe4a4c40c60a73eeaf1ff004ca1b48b77632cbabc5a5ce04cu8vr.jpg...)

Key: 007b10b45d424e7876e73b6b66d62d603c6ef2ee886761e570b2110e47608iwlr.jpg
Shape: (1737, 3088)
Data type: uint8
Array contents:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [4 4 4 ... 0 0 0]
 [4 4 4 ... 0 0 0]
 [4 4 4 ... 0 0 0]]

Key: 01hq73db2125c694f5f555f744b2020c112251215b9574bb0101ff04ae007ut37.jpg
Shape: (2208, 2944)
Data type: uint8
Array contents:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 4 4 4]
 [0 0 0 ... 4 4 4]
 [0 0 0 ... 4 4 4]]

Key: 059hc8436a3

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

filenames = npz_filenames  # npz_filenames = list(npz_panic.keys())

for filename in filenames[:30]:
    data = npz_panic[filename]  # Segmentation 데이터 로드

    # 원본 이미지 로드
    image_path = os.path.join(panic_img_path, filename)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 새로운 이미지 생성 (모든 부분을 검정색으로 초기화)
    new_img = image.copy()  # 원본 이미지 복사

    # 모든 영역을 검정색으로 초기화
    if len(image.shape) == 2:  # 단일 채널 이미지
        new_img[:, :] = 0
    else:  # RGB 이미지
        new_img[:, :] = [0, 0, 0]

    # Segmentation 데이터에서 얼굴 영역(값이 3)을 추출
    mask = data == 3
    new_img[mask] = image[mask]  # 얼굴만 원본 이미지에서 가져와서 새로운 이미지에 덮어씌움

    # 시각화
    plt.figure(figsize=(12, 6))

    # 원본 이미지
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis("off")

    # 얼굴만 원본 이미지로 남기고 나머지 부분은 검정색인 이미지 시각화
    plt.subplot(1, 2, 2)
    plt.imshow(new_img)
    plt.title("Face Only (Rest Black)")
    plt.axis("off")

    plt.suptitle(f"File : {filename}")
    plt.show()


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

# 경로 설정
data_path = '/workspace/Data_All/'
img_path = data_path + 'image/'                     # /workspace/Data_All/image/
segment_path = data_path + 'segmentation/'          # /workspace/Data_All/segmentation/

panic_img_path = img_path + 'panic/'  # 원본 이미지 경로
output_path = '/workspace/Data_Augumentation/segmentation_face/image/train/panic/'  # 결과 저장 경로

# npz 파일 로드 (파일 경로에 맞게 수정)
npz_panic = np.load(segment_path + 'combined_panic_updated.npz')  # npz 파일 경로 수정
npz_filenames = list(npz_panic.keys())  # npz 파일에서 모든 파일 이름을 리스트로 가져옴

# 첫 30개 파일을 처리
for filename in npz_filenames[:30]:
    data = npz_panic[filename]  # Segmentation 데이터 로드

    # 원본 이미지 로드
    image_path = os.path.join(panic_img_path, filename)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 새로운 이미지 생성 (모든 부분을 검정색으로 초기화)
    new_img = image.copy()  # 원본 이미지 복사

    # 모든 영역을 검정색으로 초기화
    if len(image.shape) == 2:  # 단일 채널 이미지
        new_img[:, :] = 0
    else:  # RGB 이미지
        new_img[:, :] = [0, 0, 0]

    # Segmentation 데이터에서 얼굴 영역(값이 3)을 추출
    mask = data == 3  # 얼굴이 3으로 지정되어 있다고 가정
    new_img[mask] = image[mask]  # 얼굴만 원본 이미지에서 가져와서 새로운 이미지에 덮어씌움

    # 얼굴만 원본 이미지로 남기고 나머지 부분은 검정색인 이미지를 640x640으로 리사이징 후 저장
    output_file_path = os.path.join(output_path, f"face_only_{filename}")  # 저장할 경로 설정
    new_img_bgr = cv2.cvtColor(new_img, cv2.COLOR_RGB2BGR)  # OpenCV는 BGR 포맷을 사용하므로 변환

    # 640x640으로 리사이즈
    resized_img = cv2.resize(new_img_bgr, (640, 640))

    # 리사이즈된 이미지 저장
    cv2.imwrite(output_file_path, resized_img)  # 이미지 저장

    # 시각화
    plt.figure(figsize=(12, 6))

    # 원본 이미지
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis("off")

    # 얼굴만 원본 이미지로 남기고 나머지 부분은 검정색인 이미지 시각화
    plt.subplot(1, 2, 2)
    plt.imshow(new_img)
    plt.title("Face Only (Rest Black)")
    plt.axis("off")

    plt.suptitle(f"File : {filename}")
    plt.show()


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import json

# 경로 설정
data_path = '/workspace/Data_All/'
img_path = data_path + 'image/'  # /workspace/Data_All/image/
segment_path = data_path + 'segmentation/'  # /workspace/Data_All/segmentation/

panic_img_path = img_path + 'panic/'  # 원본 이미지 경로
output_path = '/workspace/Data_Augumentation/segmentation_face/image/train/panic/'  # 결과 저장 경로
output_label_path = '/workspace/Data_Augumentation/segmentation_face/label/train/panic/'  # 레이블 저장 경로

# 저장 경로 생성
os.makedirs(output_path, exist_ok=True)
os.makedirs(output_label_path, exist_ok=True)

# npz 파일 로드
npz_panic = np.load(segment_path + 'combined_panic_updated.npz')  # npz 파일 경로 수정
npz_filenames = list(npz_panic.keys())  # npz 파일에서 모든 파일 이름을 리스트로 가져옴

# JSON 파일 로드 (panic 관련 JSON 파일 로드)
json_path = data_path + 'label/'  # JSON 경로 설정
with open(json_path + "train_panic.json", "r", encoding='euc-kr') as f:
    json_panic_train = json.load(f)

# JSON 파일에서 레이블 데이터 매핑
label_data = {item['filename']: item for item in json_panic_train}

# 레이블 데이터를 저장할 리스트 생성
all_labels = []

# 이미지 및 레이블 처리
for filename in npz_filenames[:30]:  # 첫 30개 파일 처리
    data = npz_panic[filename]  # Segmentation 데이터 로드

    # 원본 이미지 로드
    image_path = os.path.join(panic_img_path, filename)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 새로운 이미지 생성 (검정색 초기화)
    new_img = np.zeros_like(image)  # 검정색으로 초기화
    mask = data == 3  # 얼굴 영역 추출
    new_img[mask] = image[mask]  # 얼굴만 원본 이미지에서 가져와 새로운 이미지에 덮어씌움

    # 리사이즈된 이미지 생성 (얼굴만 리사이즈 후 저장)
    resized_img = cv2.resize(new_img, (640, 640))  # 리사이즈
    resized_img_bgr = cv2.cvtColor(resized_img, cv2.COLOR_RGB2BGR)  # OpenCV는 BGR 포맷 사용
    output_file_path = os.path.join(output_path, f"face_only_{filename}")
    cv2.imwrite(output_file_path, resized_img_bgr)  # 리사이즈된 이미지 저장

    # 레이블 데이터 추가 및 저장
    if filename in label_data:
        label_info = label_data[filename]
        
        # 원본 이미지 크기
        original_height, original_width = image.shape[:2]
        
        # 리사이즈된 이미지 크기
        new_width, new_height = 640, 640
        
        # 바운딩 박스 비례적으로 크기 조정
        for annot in ['annot_A', 'annot_B', 'annot_C']:
            if annot in label_info:
                box = label_info[annot]['boxes']
                box['minX'] = int(box['minX'] * (new_width / original_width))
                box['minY'] = int(box['minY'] * (new_height / original_height))
                box['maxX'] = int(box['maxX'] * (new_width / original_width))
                box['maxY'] = int(box['maxY'] * (new_height / original_height))

        label_info['resized'] = True  # 리사이즈 여부 추가 (옵션)

        # 레이블 데이터를 리스트에 추가
        all_labels.append(label_info)

    # 시각화 (선택)
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis("off")
    plt.subplot(1, 2, 2)
    plt.imshow(resized_img)
    plt.title("Face Only (Rest Black) Resized")
    plt.axis("off")
    plt.suptitle(f"File: {filename}")
    plt.show()

# 모든 레이블을 하나의 JSON 파일로 저장
output_label_file = os.path.join(output_label_path, "combined_labels.json")
with open(output_label_file, "w", encoding="utf-8") as json_file:
    json.dump(all_labels, json_file, ensure_ascii=False, indent=4)

print(f"Labels saved to: {output_label_file}")
